In [1]:
import tensorflow as tf
import numpy as np
import time
import psutil
import os
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
import tensorflow_datasets as tfds
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pandas as pd

In [9]:
# Ensure TensorFlow only uses a limited amount of GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Define batch sizes and number of runs
BATCH_SIZES = [1, 4, 8, 16, 32, 64, 100, 128]
NUM_RUNS = 10  # Runs per batch size

# Load NASNet model
base_model = NASNetMobile(weights='imagenet', input_shape=(224, 224, 3))

# Create a model that outputs intermediate layer results
layer_outputs = [layer.output for layer in base_model.layers]
model = Model(inputs=base_model.input, outputs=layer_outputs)

# Precompile TensorFlow functions for performance
@tf.function
def run_inference(input_tensor):
    return model(input_tensor)

In [10]:
# Load CIFAR-100 dataset
def load_cifar100(batch_size):
    dataset = tfds.load("cifar100", split="train", as_supervised=True)
    dataset = dataset.map(lambda x, y: (tf.image.resize(x, [224, 224]), y))  # Resize to 224x224
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [11]:
# Function to measure inference time and memory
def benchmark_model(device, batch_size, dataset_iter):
    print(f"\nRunning on {device} with batch size {batch_size}")
    results = []

    for run in range(NUM_RUNS):
        # Load Dataset
        try:
            input_tensor, _ = next(dataset_iter)  # Get real ImageNet images
        except StopIteration:
            dataset_iter = iter(dataset)  # Restart dataset if exhausted
            input_tensor, _ = next(dataset_iter)
        
        with tf.device(device):
            tensor_sizes = {}

            # Warm-up to avoid cold start latency
            _ = run_inference(input_tensor)

            start_time = time.perf_counter()

            # Measure Layer wise tensor size
            output = run_inference(input_tensor)
            
            # Measure per-layer execution time
            for idx, layer in enumerate(model.layers):
                # Calculate tensor size in MB (float32 = 4 bytes)
                tensor_size = np.prod(output[idx].shape) * 4 / (1024 ** 2)
                tensor_sizes[layer.name] = tensor_size
            
            total_time = time.perf_counter() - start_time
    
            results.append({
                "device": device,
                "batch_size": batch_size,
                "total_time": total_time
            })

    return results

## Experiment Logging

In [12]:
# Run benchmarks for CPU and GPU (if available)
devices = ["/CPU:0"]
if gpus:
    devices.append("/GPU:0")

final_results = []

for device in devices:    
    for batch_size in BATCH_SIZES:
        dataset = load_cifar100(batch_size)
        dataset_iter = iter(dataset)
        try:
            tf.profiler.experimental.start(f"logs/Image/Cache/{device[1:4]}/{batch_size}")
            final_results.extend(benchmark_model(device, batch_size, dataset_iter))
        finally:
            tf.profiler.experimental.stop()

2025-02-05 15:27:33.927462: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:27:33.931895: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-05 15:27:33.931907: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.



Running on /CPU:0 with batch size 1


2025-02-05 15:27:37.383671: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:27:37.523646: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:27:37.523965: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/Cache/CPU/1/plugins/profile/2025_02_05_15_27_37/C17586.xplane.pb
2025-02-05 15:27:37.578661: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:27:37.580863: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-


Running on /CPU:0 with batch size 4


2025-02-05 15:27:40.962839: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:27:41.101684: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:27:41.101916: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/Cache/CPU/4/plugins/profile/2025_02_05_15_27_41/C17586.xplane.pb
2025-02-05 15:27:41.167531: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:27:41.172755: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02-

...

2025-02-05 15:33:47.460893: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:33:47.986626: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:33:47.993094: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/Cache/GPU/64/plugins/profile/2025_02_05_15_33_47/C17586.xplane.pb
2025-02-05 15:33:48.709139: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:33:48.735027: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-02


Running on /GPU:0 with batch size 100


2025-02-05 15:35:27.840336: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:35:29.185289: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:35:29.190676: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/Cache/GPU/100/plugins/profile/2025_02_05_15_35_29/C17586.xplane.pb
2025-02-05 15:35:30.415630: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-02-05 15:35:30.518263: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-0


Running on /GPU:0 with batch size 128


2025-02-05 15:37:26.411038: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-02-05 15:37:27.972349: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-02-05 15:37:27.978163: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: logs/Image/Cache/GPU/128/plugins/profile/2025_02_05_15_37_27/C17586.xplane.pb


## Results

In [13]:
df = pd.DataFrame(final_results)
df.head()

,device,batch_size,total_time
0,/CPU:0,1,0.147739
1,/CPU:0,1,0.044316
2,/CPU:0,1,0.042249
3,/CPU:0,1,0.041992
4,/CPU:0,1,0.041455


In [14]:
df.groupby(['device', 'batch_size']).agg({'total_time': ['min', 'mean', 'var', 'max']})

total_time                                
                         min      mean        var        max
device batch_size                                           
/CPU:0 1            0.041455  0.052890   0.001111   0.147739
       4            0.060602  0.066622   0.000031   0.076599
       8            0.085620  0.095859   0.000141   0.123743
       16           0.166901  0.195135   0.001677   0.278784
       32           0.453027  0.709789   0.021931   0.931951
       64           1.487445  1.825609   0.033425   2.031429
       100          2.220175  2.526821   0.037966   2.812151
       128          3.296475  3.854590   0.283979   4.928806
/GPU:0 1            0.085628  0.331242   0.536030   2.413133
       4            0.086944  0.251754   0.249392   1.672731
       8            0.087011  0.531176   1.924538   4.479369
       16           0.090890  0.786182   2.738114   5.206036
       32           0.129955  0.731244   2.596973   5.290575
       64           0.879107  2.000703   2.811939   6.419471
       100          2.202386  2.977277   0.368923   4.232437
       128          3.114430  5.992218  19.552280  16.222515